<a href="https://colab.research.google.com/github/ruperty/colab/blob/master/basic-cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install x11-utils  > /dev/null 2>&1

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 209 kB of archives.
After this operation, 711 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Fetched 209 kB in 1s (355 kB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Reading database ... 146795 files and directories currently installed.)
Preparing to unpack .../libxxf86dga1_2%3a1.1.4-1_amd64.deb ...
Unpacking libxxf86dga1:amd64 (2:1.1.4-1) ...
Selecting previously unselected package x11-utils.
Preparing to unpack .../x11-utils_7.7+3build1_amd64.de

Cartpole

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [53]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start() 

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1017'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1017'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                
                 controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [92]:
env = wrap_env(gym.make("CartPole-v1"))
observation = env.reset()
#print(observation)
loops = 100
for i in range(loops):
   env.render()
   action = env.action_space.sample()
   observation =env.step(action) # take a random action
   #print(action, observation)
   done = observation[2]
   if done: 
        if i < loops :
          print("Failed after %d loops with a pole angle of %2.2f degrees" %( i, np.rad2deg(observation[0][2])))
        else:
          print("Finished after ", i, " loops")
        env.reset()
        break

env.close()
show_video()

Failed after 25 loops with a pole angle of 12.57 degrees


In [0]:
def proportional(r, p, g):
    return (r-p)*g

def controller(ctr,     pole_angle_ref, cart_position, cart_velocity, pole_angle, pole_velocity):
    
    pole_velocity_ref=proportional(pole_angle_ref, pole_angle, pole_angle_gain)
    cart_position_ref=proportional(pole_velocity_ref, pole_velocity, pole_velocity_gain)
    cart_velocity_ref=proportional(cart_position_ref, cart_position, cart_position_gain)
    power=proportional(cart_velocity_ref, cart_velocity, cart_velocity_gain)
    
    if power>=0:
        return 0
    return 1

def get_obs(observation):
    cart_position = observation[0]
    cart_velocity = observation[1]
    pole_angle = observation[2]
    pole_velocity = observation[3]
    return cart_position,    cart_velocity ,    pole_angle , pole_velocity 


In [0]:
reward = 0
pole_angle_ref = 0

loops = 2000
pole_angle_gain = 1.8
pole_velocity_gain = 1.8
cart_position_gain = .5
cart_velocity_gain = .001

In [91]:
env = wrap_env(gym.make("CartPole-v1"))
env._max_episode_steps = 4000
observation = env.reset()
cart_position = observation[0]
cart_velocity = observation[1]
pole_angle = observation[2]
pole_velocity = observation[3]
for ctr in range(loops):
    env.render()
    #action= env.action_space.sample()
    #print(action)
    
    action = controller(ctr, pole_angle_ref, cart_position, cart_velocity, pole_angle, pole_velocity)
    
    observation =env.step(action)    
    #print(observation)
    done = observation[2]
    reward+=observation[1]
    cart_position, cart_velocity, pole_angle, pole_velocity = get_obs(observation[0])
    #print(cart_position)
    if done: 
        print(observation)
        break
 
print("Reward ", reward)    

env.close()
show_video()

(array([ 0.01621908,  0.04037531,  0.00749049, -0.06127766]), 1.0, True, {'TimeLimit.truncated': True})
Reward  500.0
